## Import Libraries

In [56]:
import os
import sys

## Read text from PDF
##### Provide the path of PDF file before initialized everything

In [57]:
data='data.pdf'

In [58]:
data

'data.pdf'

## Extract data by reading

In [59]:
import PyPDF2
from docx import Document

In [60]:
def extract_text_pdf(pdf):
    text = ""
    reader = PyPDF2.PdfReader(pdf)
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

In [61]:
def extract_text_docx(docx):
    doc = Document(docx)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

In [62]:
'''def get_pdf_text(pdf_docs):
    text =""
    for pdf in pdf_docs:
        pdf_reader =PyPDF2.PdfReader(pdf_docs) 
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text'''



'''The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'''


'The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'

In [63]:
def extract(doc):
    _, file_extension = doc.split('.')
    print(file_extension)
    if file_extension == 'pdf':
        return extract_text_pdf(doc)
    elif file_extension == 'docx':
        return extract_text_docx(doc)
    else:
        raise ValueError("Unsupported file format")

In [64]:
text_data=extract(data)


pdf


In [65]:
#print(text_data)

### Convert each text into chunks
##### need to split the text using CharacterTextSplitter (which is basically present in Langchain.text_splitter) such that it should not increase token size


In [66]:
from langchain.text_splitter import CharacterTextSplitter

In [67]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size =1300,
    chunk_overlap =300,
    length_function =len,

    )

In [68]:
chunks = text_splitter.split_text(text_data)

#taking the particular text & splitting based on information

In [69]:
len(chunks)

61

In [70]:
print(chunks[12])

cal hrs.)  Introduction  to System  Development  environment,  The Object  paradigm,  Objec - 
tory  S/W  development  process,  Object  Oriented  Modelling  using  UML,  Use case  diagrams, 
Class  diagram,  Interaction  diagram, Class  declaration, Method  implementation, Informa - 
tion hiding and Access modifiers, Class Hierarchy (inheritance), Polymorphism, Packages 
and Data  abstraction.  Practical  using  Rational  Rose  and OO Programming  using  Java/C++ 
Evaluatio n Method:  Practical  Examination  30%  and End Semester  Examina - 
tion 70% (Student must obtain at least 35% from the theory paper and the 30% 
from the practical paper)  7  COM213 α: Operating  Systems  (15 lecture  hrs. & 22.5  practical  hrs.)   Introduc - 
tion, Process Management, CPU Scheduling, Deadlocks, Memory Management, File -System 
Implementation, I/O Systems.  Practical oriented  
Evaluation Method:  Practical Examination 30% and End  Semester Examina - 
tion 70% (Student must obtain at least 35% f

In [71]:
#print(chunks)

## Convert chunks into vectors(Text Embeddings)

In [72]:
import os
#Import 'OpenAIEmbeddings class'
from langchain.embeddings import OpenAIEmbeddings

from langchain_community.vectorstores import FAISS
#initialize the connection of my db

# Load environment variables from the .env file
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv()) 

# Retrieve the OpenAI API key from the environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Ensure the API key is available
if OPENAI_API_KEY is None:
    raise ValueError("OpenAI API key not found. Please set it in the .env file.")

# Initialize OpenAIEmbeddings with your API key
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)



In [73]:
#store in the vector db 
db = FAISS.from_texts(texts = chunks,embedding = embedding)

In [74]:
if os.path.exists("vectorstore/index.pkl"):
    # Load the existing vector store
    db = FAISS.load_local("vectorstore", embedding,allow_dangerous_deserialization=True)
else:
    # Create a new vector store from the text chunks
    db = FAISS.from_texts(texts=chunks, embedding=embedding)
    # Save the newly created vector store locally
    db.save_local("vectorstore")

### Similarity Check

In [75]:
#query = "what are the Minimum Requirement for the Completion of the Bachelor of Computer Science Honours Degree ?"
query = "who are the lecturers in computer science department with their designation and their qualifications"
docs = db.similarity_search(query)

In [76]:
len(docs)

4

In [77]:
print(docs[0])

page_content='Lecturer  
 
 
 
 
 
 
 
Probationary 
Lecturer  Dr. W. A. Mohotti  
B.Sc.  IT (Mo ratuwa, SL) 
M.Sc.  in IT (Moratuwa,  SL) 
Ph.D.  (QUT,  Australia)  Data  Mining  and Machine  Learning,  
Text Clustering, Outlier Detection, and 
Cluster Evolution, Social Media Analytics  
Dr. D. W. C. P. Kumari  
B. Sc.  in Comp.  Sci. (UCSC, SL) 
M.Phil.  (UCSC, SL)  
PhD (QUT, Australia) 
(On Leave)  
Mr. K. D. C. G. Kapugama 
BCS (Ruhuna, SL)  
Reading  for PhD  (Monash,  Australia)  
(On Leave)  Information  Security,  Process  
Analytics, Mathematical 
Modelling,  Event  log analysis 
Data Structure & Algorithms 
Evolutionary Algorithms  
Data Mining, Text Mining  
Mr. P. D. T. Chathuranga  
BCS (Ruhuna,  SL) Natural  Language  Processing,  
Sentiment  Analysis,  Text  Mining,  
Machine  Learning  
Mr. L.L. Gihan  Chathuranga  
B.Sc  (Sabaragamuwa,  S.L) Machine  Learning,  Artificial  Neural  
Networks,  Artificia l Intelligence,  
Data  mining  
Ms. H.D.  Supuni  Shashikala  
B.

In [78]:
docs = db.similarity_search_with_score(query)

In [79]:
docs[0]

(Document(page_content='Lecturer  \n \n \n \n \n \n \n \nProbationary \nLecturer  Dr. W. A. Mohotti  \nB.Sc.  IT (Mo ratuwa, SL) \nM.Sc.  in IT (Moratuwa,  SL) \nPh.D.  (QUT,  Australia)  Data  Mining  and Machine  Learning,  \nText Clustering, Outlier Detection, and \nCluster Evolution, Social Media Analytics  \nDr. D. W. C. P. Kumari  \nB. Sc.  in Comp.  Sci. (UCSC, SL) \nM.Phil.  (UCSC, SL)  \nPhD (QUT, Australia) \n(On Leave)  \nMr. K. D. C. G. Kapugama \nBCS (Ruhuna, SL)  \nReading  for PhD  (Monash,  Australia)  \n(On Leave)  Information  Security,  Process  \nAnalytics, Mathematical \nModelling,  Event  log analysis \nData Structure & Algorithms \nEvolutionary Algorithms  \nData Mining, Text Mining  \nMr. P. D. T. Chathuranga  \nBCS (Ruhuna,  SL) Natural  Language  Processing,  \nSentiment  Analysis,  Text  Mining,  \nMachine  Learning  \nMr. L.L. Gihan  Chathuranga  \nB.Sc  (Sabaragamuwa,  S.L) Machine  Learning,  Artificial  Neural  \nNetworks,  Artificia l Intelligence,  \nDa

In [80]:
docs[1]

(Document(page_content='B.Sc.  (UWU,  S.L.),  \nReading  for M.Sc.  (Peradeniya,  S.L.)  Machine  Learning,  Feature  Extraction,  \nDeep  Learning  \n1.4 Members  of the Academic  Support  Staff  \n \nAcademic  Supportive  Staff  Members  \nDesignation  Name  \nProgrammer  Cum  Systems  Analyst  Mr. H. G. U. Harankahadeniya  \nMr. B. H. Saranapala  \nMr. A. P. Luwishewa  \nInstructor  in Computer  Technology  Ms. P. B. N. K. De Silva  \nMr. U. V. Malawara  Arachchi \nMs. W.P. Priyanthi  \nMr. R. Wickramaratne \nMr. C.L. Wimalaratne \nMs. G.K. Mabula  \nMr. L.W.  Wellakkage \nMr. G.M.T. Ranjana \nMrs.  W. K. Shajith  \nMiss.  H. G. S. Priyangani  \n \n \n1.5 Course  Units  in Computer  Science  for B.Sc.  (General)  Degree  \nLevel  I - Semester  I \nCOM1111:   Basic  Concepts  of Information  Technology  (15 lecture  hrs.)    Overv iew \nof Computer System, Function of Computer System, Input/Output Peripherals, Computer \nStorages, Systems Software, Data type and Data representation, 

In [81]:
docs[2]

(Document(page_content='Rule Modeling  \n• Network  Monitor ing and Acquiring  and Managing  Information  \n• E-commerce information systems development, Model -drivers design, Goal, Business  \n& service Modelling  \n• Computational Geometry, Computer Graphics programming, Design and Analysis of \nAlgorithms, Graph Theory  \n• Computati onal Systems Biology, Bioinformatics, Modelling and Simulation, Neural \nComputing, stochastic modelling  \n• Embedded  Systems,  reconfigurable  computing,  Bioinformatics  \n• Knowledge Representation, Ontology, Semantic Web, Ontology Engineering, Mobile \nApplications  4  1.2 Head of the Department  \nDr. W.A.  Indika  \nB.Sc.  (Kelaniya,  S.L.),  M.Sc.  (Kelaniya,  S.L.),  PhD  (UCSC,  S.L.)  \n \n1.3 Members  of Academic  Staff  \n \n \nDesignation  Name  Specialization  \nSenior  \nLecturer  Mr. S. A. S. Lorensuhewa  \nB.Sc.  (Colombo, SL) \nM.Sc.  (Zhejiang,  China)  Comput er Applications,  Text  Mining  and \nText Classification Data Mining, \

In [82]:
docs[3]

(Document(page_content='the Bachelor of Computer Science (Honours) Degree courses.  \nThe Honours degree consists of examinations at the end of each semester or at the end  \nof course unit for research project.  \n \n1.9.1  Attendance  \nTo be eligible to sit for an examination of a theory or practical course unit, there should \nminimum attendance of 80%.  If a student fails to meet this requirement for a particular \ncourse  unit, he  or she will be considered  to have  failed  in that  course  unit (see  section  14.3.1 \nfor further details).  \n \n1.9.2  Examination Criteria \nEvaluation Methods  \n• Theory  examination  (written/oral)  \n• Practical  examination  \n• Continuous  assessment  \n• Assignment  \n• Report  \n• Presentation  \n \nGrading  Syst em \nAwarding of grades for course units will be done according to the grading system given in  \nthe Table in section 14.2  \n \nPass  in a Course  Unit  \n• A candidate  who obtains grade  C or  better  for a Course  Unit  wil

In [83]:
#docs[4]

### Similarity Check by Vector

In [84]:
embedding_vector= embedding.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Lecturer  
 
 
 
 
 
 
 
Probationary 
Lecturer  Dr. W. A. Mohotti  
B.Sc.  IT (Mo ratuwa, SL) 
M.Sc.  in IT (Moratuwa,  SL) 
Ph.D.  (QUT,  Australia)  Data  Mining  and Machine  Learning,  
Text Clustering, Outlier Detection, and 
Cluster Evolution, Social Media Analytics  
Dr. D. W. C. P. Kumari  
B. Sc.  in Comp.  Sci. (UCSC, SL) 
M.Phil.  (UCSC, SL)  
PhD (QUT, Australia) 
(On Leave)  
Mr. K. D. C. G. Kapugama 
BCS (Ruhuna, SL)  
Reading  for PhD  (Monash,  Australia)  
(On Leave)  Information  Security,  Process  
Analytics, Mathematical 
Modelling,  Event  log analysis 
Data Structure & Algorithms 
Evolutionary Algorithms  
Data Mining, Text Mining  
Mr. P. D. T. Chathuranga  
BCS (Ruhuna,  SL) Natural  Language  Processing,  
Sentiment  Analysis,  Text  Mining,  
Machine  Learning  
Mr. L.L. Gihan  Chathuranga  
B.Sc  (Sabaragamuwa,  S.L) Machine  Learning,  Artificial  Neural  
Networks,  Artificia l Intelligence,  
Data  mining  
Ms. H.D.  Supuni  Shashikala  
B.Sc.  (UWU,  S.

In [85]:
print(docs[2])

page_content='Rule Modeling  
• Network  Monitor ing and Acquiring  and Managing  Information  
• E-commerce information systems development, Model -drivers design, Goal, Business  
& service Modelling  
• Computational Geometry, Computer Graphics programming, Design and Analysis of 
Algorithms, Graph Theory  
• Computati onal Systems Biology, Bioinformatics, Modelling and Simulation, Neural 
Computing, stochastic modelling  
• Embedded  Systems,  reconfigurable  computing,  Bioinformatics  
• Knowledge Representation, Ontology, Semantic Web, Ontology Engineering, Mobile 
Applications  4  1.2 Head of the Department  
Dr. W.A.  Indika  
B.Sc.  (Kelaniya,  S.L.),  M.Sc.  (Kelaniya,  S.L.),  PhD  (UCSC,  S.L.)  
 
1.3 Members  of Academic  Staff  
 
 
Designation  Name  Specialization  
Senior  
Lecturer  Mr. S. A. S. Lorensuhewa  
B.Sc.  (Colombo, SL) 
M.Sc.  (Zhejiang,  China)  Comput er Applications,  Text  Mining  and 
Text Classification Data Mining, 
Rule Extraction and  
Knowledge 

In [86]:
print(docs[3])

page_content='the Bachelor of Computer Science (Honours) Degree courses.  
The Honours degree consists of examinations at the end of each semester or at the end  
of course unit for research project.  
 
1.9.1  Attendance  
To be eligible to sit for an examination of a theory or practical course unit, there should 
minimum attendance of 80%.  If a student fails to meet this requirement for a particular 
course  unit, he  or she will be considered  to have  failed  in that  course  unit (see  section  14.3.1 
for further details).  
 
1.9.2  Examination Criteria 
Evaluation Methods  
• Theory  examination  (written/oral)  
• Practical  examination  
• Continuous  assessment  
• Assignment  
• Report  
• Presentation  
 
Grading  Syst em 
Awarding of grades for course units will be done according to the grading system given in  
the Table in section 14.2  
 
Pass  in a Course  Unit  
• A candidate  who obtains grade  C or  better  for a Course  Unit  will be considered  to 
have passed i

In [87]:
#print(docs[4])

In [88]:
most_similar_doc = docs[0]

### Generate the Answer

In [89]:
from langchain.llms import OpenAI
llm = OpenAI(api_key = OPENAI_API_KEY)

In [90]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm = llm , chain_type = 'stuff')

In [91]:
response = chain.run(input_documents = docs , question = query)

In [92]:
response

'\nThe lecturers in the computer science department are listed below with their designation and qualifications:\n\n1. Lecturer - Dr. W.A. Indika (Head of the Department)\nB.Sc. (Kelaniya, S.L.), M.Sc. (Kelaniya, S.L.), PhD (UCSC, S.L.)\n\n2. Senior Lecturer - Mr. S.A.S. Lorensuhewa\nB.Sc. (Colombo, SL), M.Sc. (Zhejiang, China)\n\n3. Probationary Lecturer - Dr. W.A. Mohotti\nB.Sc. IT (Moratuwa, SL), M.Sc. in IT (Moratuwa, SL), PhD (QUT, Australia)\n\n4. Probationary Lecturer - Dr. D.W.C.P. Kumari\nB.Sc. in Comp. Sci. (UCSC, SL), M.Phil. (UCSC, SL), PhD (QUT, Australia)\n\n5. Probationary Lecturer - Mr. K.D.C.G. Kapugama\nBCS (Ruhuna, SL), Reading for PhD (Monash, Australia)\n\n6. Probationary Lecturer - Mr. P.D.T. Chathuranga\nBCS'

In [93]:
print(response)


The lecturers in the computer science department are listed below with their designation and qualifications:

1. Lecturer - Dr. W.A. Indika (Head of the Department)
B.Sc. (Kelaniya, S.L.), M.Sc. (Kelaniya, S.L.), PhD (UCSC, S.L.)

2. Senior Lecturer - Mr. S.A.S. Lorensuhewa
B.Sc. (Colombo, SL), M.Sc. (Zhejiang, China)

3. Probationary Lecturer - Dr. W.A. Mohotti
B.Sc. IT (Moratuwa, SL), M.Sc. in IT (Moratuwa, SL), PhD (QUT, Australia)

4. Probationary Lecturer - Dr. D.W.C.P. Kumari
B.Sc. in Comp. Sci. (UCSC, SL), M.Phil. (UCSC, SL), PhD (QUT, Australia)

5. Probationary Lecturer - Mr. K.D.C.G. Kapugama
BCS (Ruhuna, SL), Reading for PhD (Monash, Australia)

6. Probationary Lecturer - Mr. P.D.T. Chathuranga
BCS
